<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Graph_Neural_Networks_(GNNs).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def normalize_adjacency_matrix(adj):
    D = torch.diag(torch.pow(adj.sum(dim=1), -0.5))
    return torch.mm(torch.mm(D, adj), D)

class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x, adjacency_matrix):
        support = torch.mm(adjacency_matrix, x)  # Aggregate features using adjacency matrix
        output = self.linear(support)  # Apply linear transformation
        return output

class GCN(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super(GCN, self).__init__()
        self.gcn1 = GCNLayer(in_features, hidden_features)
        self.gcn2 = GCNLayer(hidden_features, out_features)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x, adjacency_matrix):
        x = self.dropout(F.relu(self.gcn1(x, adjacency_matrix)))  # First GCN layer with dropout
        x = F.log_softmax(self.gcn2(x, adjacency_matrix), dim=1)  # Second GCN layer with softmax
        return x

# Example usage
nodes = torch.randn(4, 16)  # 4 nodes, 16 features per node
adjacency_matrix = torch.tensor(
    [[1, 1, 0, 0],
     [1, 1, 1, 0],
     [0, 1, 1, 1],
     [0, 0, 1, 1]], dtype=torch.float32
)  # Example adjacency matrix with edges

normalized_adjacency_matrix = normalize_adjacency_matrix(adjacency_matrix)  # Normalize adjacency matrix

model = GCN(in_features=16, hidden_features=32, out_features=2)
output = model(nodes, normalized_adjacency_matrix)
print(output.shape)  # Output should have shape (4, 2)